In [1]:
#Import the dependencies for this workflow
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nupack
import pickle
from tkinter import Tk
from tkinter.filedialog import askdirectory
from nupack import * # This star means import all

print(nupack.__version__)

4.0.0


In [2]:
#Toehold Switch VISTA Design Code 

# Step 1: Create toehold switches tiled across source sequence inputs, at specified temperature, with output reporter sequence
# VISTA desgin code is based on tsgen2 toehold switch architechure. Nucleotides 7-18 of the ascending stem, ...
# the loop, and nucleotides of the descending stem, including the start codon AUG are conserved. The sequence for this conserved element is: GUUAUAGUUAUGAACAGAGGAGACAUAACAUGAAC
# For these desings, a 36-nt toehold is used, with an unwinding of 6-bp in the stem. 

class toehold_switch_design_object():

    # The initialization step takes in a file called 'source_seq_list.csv' and saves info uniquely to this instance of the toehold design class
    # Parameters will change depending on the design specified in 'source_seq_list.csv'
    # The column names in 'source_seq_list.csv' must remain constant. 
    def __init__(self,source_seq_list,ecoli_codon_usage_table,target_type,num_designs):
        self.path = askdirectory(title='Select Folder')
        os.chdir(self.path)
        input_table = pd.read_csv(source_seq_list)
        self.mRNA_names0 = input_table[['Gene Name']]
        self.mRNA_sequences0 = input_table[['Gene Sequence']]
        self.temp = input_table[['Temperature']]
        self.output_name = input_table[['Output Name']]
        self.output_sequence = input_table[['Output Sequence']]
        self.hairpin_seq_top = 'GUUAUAGUUAUGAACAGAGGAGACAUAACAUGAAC' #top part of the toehold switch stem/loop, including RBS 
        self.hairpin_suffix = 'AACCUGGCGGCAGCGCAAAAG' #linker sequence between toehold switch & reporter output 
        mRNA_names = {}; mRNA_sequences = {}; mRNA_temps = [];
        self.num_designs = num_designs

        self.design_type = 'Toehold_VISTA';
        os.mkdir(self.design_type);
        self.target_type = target_type;
        self.codon_usage_df = pd.read_csv(ecoli_codon_usage_table); self.codon_usage_dict = dict(zip(self.codon_usage_df["Codon"], self.codon_usage_df["Fraction"]))

    def reverse_complement_seq(self,nucleotide_sequence):
        """
        Calculate the reverse complement of an RNA sequence.
        Args: rna_sequence (str):  A nucleotide sequence consisting of 'A', 'T'/'U', 'C', 'G'.
        Returns: str: The reverse complement of the RNA sequence.
        """
        nucleotide_sequence = nucleotide_sequence.upper()
        # Define the complement mapping for RNA
        complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
        # Reverse the sequence and create its complement
        reverse_sequence = nucleotide_sequence[::-1]
        reverse_complement = [complement[base] for base in reverse_sequence]
        # Join the complement bases to form the reverse complement sequence
        reverse_complement_sequence = "".join(reverse_complement)

        return reverse_complement_sequence

    def dna_to_rna(self,dna_sequence):
        dna_sequence = dna_sequence.upper() 
        return dna_sequence.replace('T', 'U')

    def rna_to_dna(self,dna_sequence):
        dna_sequence = dna_sequence.upper() 
        return dna_sequence.replace('U', 'T')

    def nt_to_aa(self,seq):
        """Convert nucleotide sequence to amino acids."""
        codon_table = {
            'UUU':'F', 'UUC':'F', 'UUA':'L', 'UUG':'L','UCU':'S', 'UCC':'S', 'UCA':'S', 'UCG':'S',
            'UAU':'Y', 'UAC':'Y', 'UAA':'*', 'UAG':'*','UGU':'C', 'UGC':'C', 'UGA':'*', 'UGG':'W',
            'CUU':'L', 'CUC':'L', 'CUA':'L', 'CUG':'L','CCU':'P', 'CCC':'P', 'CCA':'P', 'CCG':'P',
            'CAU':'H', 'CAC':'H', 'CAA':'Q', 'CAG':'Q','CGU':'R', 'CGC':'R', 'CGA':'R', 'CGG':'R',
            'AUU':'I', 'AUC':'I', 'AUA':'I', 'AUG':'M','ACU':'T', 'ACC':'T', 'ACA':'T', 'ACG':'T',
            'AAU':'N', 'AAC':'N', 'AAA':'K', 'AAG':'K','AGU':'S', 'AGC':'S', 'AGA':'R', 'AGG':'R',
            'GUU':'V', 'GUC':'V', 'GUA':'V', 'GUG':'V','GCU':'A', 'GCC':'A', 'GCA':'A', 'GCG':'A',
            'GAU':'D', 'GAC':'D', 'GAA':'E', 'GAG':'E','GGU':'G', 'GGC':'G', 'GGA':'G', 'GGG':'G'
        }
        aa_seq = ""
        for i in range(0, len(seq)-2, 3):
            codon = seq[i:i+3]
            aa_seq += codon_table.get(codon, 'X')  # 'X' for unknown codons
        return aa_seq

    def compute_sensor_parameters(self,sensor_sequence_r):
        """
        Calculates minimum free energy (MFE) and ensemble defects of various regions of the toehold switch sensor. 
        Args: sensor_sequence_r (str):  An RNA sequence consisting of 'A', 'U', 'C', 'G'.
        Returns: appended columns to output_table: A dictionary containing computed thermodynamic values for toehold switch sensor. 
        """
        #Define the ideal structure for various regions of the toehold switch sensor; IMPORTANT: these are defined based on the TSgen2 Toehold Switch Structure, where the toehold is 30nt, and the target invades 6nt into the stem. 
        AS = sensor_sequence_r[30:48]; AS_struc = '.' * len(AS); #ascending stem
        DS = sensor_sequence_r[58:77]; DS_struc = '.' * len(DS); #descending stem
        RBS_GFP = sensor_sequence_r[48:116]; RBS_GFP_struc = 'U%s D%s U%s U%s'%(len(self.hairpin_seq_top[12:-3]),(self.len_d+self.len_a),(self.len_b+self.len_a),(self.hairpin_suffix_len+len(self.output_seq30[:9])));
        RBS_Linker = sensor_sequence_r[48:107]; RBS_Linker_struc = 'U%s D%s U%s U%s'%(len(self.hairpin_seq_top[12:-3]),(self.len_d+self.len_a),(self.len_b+self.len_a),self.hairpin_suffix_len);
        Switch_Off = sensor_sequence_r[:77]; Switch_Off_struc = 'U%s D%s D3 (U3 D6 U11 U3)'%(self.len_c,(self.len_a+self.len_b));
        Switch_Off_GFP = sensor_sequence_r[:116]; Switch_Off_GFP_struc = 'U%s D%s D3 (U3 D6 U11 U3) U%s U%s'%(self.len_c,(self.len_a+self.len_b),(self.len_d+self.len_a+3),(self.hairpin_suffix_len+len(self.output_seq30[:9])));
        Switch_Off_NoTH = sensor_sequence_r[30:116]; Switch_Off_NoTH_struc = 'D%s D3 (U3 D6 U11 U3) U%s U%s'%((self.len_a+self.len_b),(self.len_d+self.len_a+3),(self.hairpin_suffix_len+len(self.output_seq30[:9])));
        Hairpin = sensor_sequence_r[30:77]; Hairpin_struc = 'D%s D3 (U3 D6 U11 U3)'%(self.len_a+self.len_b);
        Linker_GFP = sensor_sequence_r[77:116]; TH_Linker_struc = 'U%s + U%s'%(self.len_c,(self.len_a+self.len_b+self.len_d+self.hairpin_suffix_len+len(self.output_seq30[:9])));TH = sensor_sequence_r[:30];
        Switch_On = sensor_sequence_r[:76]; Switch_On_struc = 'D%s + U%s'%((self.len_c+self.len_a+self.len_b),(len(self.hairpin_seq_top)+self.len_a+self.len_b))
        Switch_On_GFP_struc = 'D%s + U%s D6 U11 U3 D%s U%s U%s'%((self.toehold_len+self.len_a+self.len_b),(self.len_a+self.len_b), (3+self.len_a), (self.len_b+self.len_d), (self.hairpin_suffix_len+len(self.output_seq30[:9])))
        
        #Calcualte minimum free energy (MFE) parameters
        mfe_structureAS = mfe(strands=[AS], model=self.my_model); self.output_table[f"Ascending Stem MFE"].append(mfe_structureAS[0].energy);
        mfe_structureDS = mfe(strands=[DS], model=self.my_model); self.output_table[f"Descending Stem MFE"].append(mfe_structureDS[0].energy);
        mfe_structureRBS_GFP = mfe(strands=[RBS_GFP], model=self.my_model); self.output_table[f"RBS-GFP Stem MFE"].append(mfe_structureRBS_GFP[0].energy);
        mfe_structureRBS_Linker = mfe(strands=[RBS_Linker], model=self.my_model); self.output_table[f"RBS-Linker MFE"].append(mfe_structureRBS_Linker[0].energy);
        mfe_structureSwitch_Off = mfe(strands=[Switch_Off], model=self.my_model); self.output_table[f"Switch Off MFE"].append(mfe_structureSwitch_Off[0].energy);
        mfe_structureSwitch_Off_GFP = mfe(strands=[Switch_Off_GFP], model=self.my_model); self.output_table[f"Switch Off GFP MFE"].append(mfe_structureSwitch_Off_GFP[0].energy);
        mfe_structureSwitch_Off_NoTH = mfe(strands=[Switch_Off_NoTH], model=self.my_model); self.output_table[f"Switch Off NoTH MFE"].append(mfe_structureSwitch_Off_NoTH[0].energy);
        mfe_structureHairpin = mfe(strands=[Hairpin], model=self.my_model); self.output_table[f"Hairpin MFE"].append(mfe_structureHairpin[0].energy);
        mfe_structureTH_Linker = mfe(strands=[TH, Linker_GFP], model=self.my_model); self.output_table[f"TH+Linker/GFP MFE"].append(mfe_structureTH_Linker[0].energy);
        mfe_structureSwitch_On = mfe(strands=[self.target_sequence,Switch_Off], model=self.my_model); self.output_table[f"Switch ON MFE"].append(mfe_structureSwitch_On[0].energy);
        mfe_structureSwitch_On_GFP = mfe(strands=[self.target_sequence,Switch_Off_GFP], model=self.my_model); self.output_table[f"Switch ON-GFP MFE"].append(mfe_structureSwitch_On_GFP[0].energy);
    
        #Calcualte ideal ensemble defect, or the defect from the specified structure of the toehold switch.
        ensemble_defect_idealAS = defect(strands=[AS], structure= AS_struc, model=self.my_model); self.output_table[f"Ascending Stem IED"].append(ensemble_defect_idealAS)
        ensemble_defect_idealDS = defect(strands=[DS], structure= DS_struc, model=self.my_model); self.output_table[f"Descending Stem IED"].append(ensemble_defect_idealDS)
        ensemble_defect_idealRBS_GFP = defect(strands=[RBS_GFP], structure=RBS_GFP_struc, model=self.my_model); self.output_table[f"RBS-GFP Stem IED"].append(ensemble_defect_idealRBS_GFP)
        ensemble_defect_idealRBS_Linker = defect(strands=[RBS_Linker], structure=RBS_Linker_struc, model=self.my_model); self.output_table[f"RBS-Linker IED"].append(ensemble_defect_idealRBS_Linker)
        ensemble_defect_idealSwitch_Off = defect(strands=[Switch_Off], structure=Switch_Off_struc, model=self.my_model); self.output_table[f"Switch Off IED"].append(ensemble_defect_idealSwitch_Off)
        ensemble_defect_idealSwitch_Off_GFP = defect(strands=[Switch_Off_GFP], structure=Switch_Off_GFP_struc, model=self.my_model); self.output_table[f"Switch Off GFP IED"].append(ensemble_defect_idealSwitch_Off_GFP)
        ensemble_defect_idealSwitch_Off_NoTH = defect(strands=[Switch_Off_NoTH], structure=Switch_Off_NoTH_struc, model=self.my_model); self.output_table[f"Switch Off NoTH IED"].append(ensemble_defect_idealSwitch_Off_NoTH)
        ensemble_defect_idealHairpin = defect(strands=[Hairpin], structure=Hairpin_struc, model=self.my_model); self.output_table[f"Hairpin IED"].append(ensemble_defect_idealHairpin)
        ensemble_defect_idealTH_Linker = defect(strands=[TH,Linker_GFP], structure=TH_Linker_struc, model=self.my_model); self.output_table[f"TH+Linker/GFP IED"].append(ensemble_defect_idealTH_Linker)
        ensemble_defect_idealSwitch_On = defect(strands=[self.target_sequence,Switch_Off], structure=Switch_On_struc, model=self.my_model); self.output_table[f"Switch ON IED"].append(ensemble_defect_idealSwitch_On)
        ensemble_defect_idealSwitch_On_GFP = defect(strands=[self.target_sequence,Switch_Off_GFP], structure=Switch_On_GFP_struc, model=self.my_model); self.output_table[f"Switch ON-GFP IED"].append(ensemble_defect_idealSwitch_On_GFP)
        
        #Calculate native ensemble defect, or the defect from the MFE proxy structure calculated earlier in this fuction. 
        ensemble_defect_nativeAS = defect(strands=[AS], structure=str(mfe_structureAS[0].structure), model=self.my_model); self.output_table[f"Ascending Stem NED"].append(ensemble_defect_nativeAS)
        ensemble_defect_nativeDS = defect(strands=[DS], structure=str(mfe_structureDS[0].structure), model=self.my_model); self.output_table[f"Descending Stem NED"].append(ensemble_defect_nativeDS)
        ensemble_defect_nativeRBS_GFP = defect(strands=[RBS_GFP], structure=str(mfe_structureRBS_GFP[0].structure), model=self.my_model); self.output_table[f"RBS-GFP Stem NED"].append(ensemble_defect_nativeRBS_GFP)
        ensemble_defect_nativeRBS_Linker = defect(strands=[RBS_Linker], structure=str(mfe_structureRBS_Linker[0].structure), model=self.my_model); self.output_table[f"RBS-Linker NED"].append(ensemble_defect_nativeRBS_Linker)
        ensemble_defect_nativeSwitch_Off = defect(strands=[Switch_Off], structure=str(mfe_structureSwitch_Off[0].structure), model=self.my_model); self.output_table[f"Switch Off NED"].append(ensemble_defect_nativeSwitch_Off)
        ensemble_defect_nativeSwitch_Off_GFP = defect(strands=[Switch_Off_GFP], structure=str(mfe_structureSwitch_Off_GFP[0].structure), model=self.my_model); self.output_table[f"Switch Off GFP NED"].append(ensemble_defect_nativeSwitch_Off_GFP)
        ensemble_defect_nativeSwitch_Off_NoTH = defect(strands=[Switch_Off_NoTH], structure=str(mfe_structureSwitch_Off_NoTH[0].structure), model=self.my_model); self.output_table[f"Switch Off NoTH NED"].append(ensemble_defect_nativeSwitch_Off_NoTH)
        ensemble_defect_nativeHairpin = defect(strands=[Hairpin], structure=str(mfe_structureHairpin[0].structure), model=self.my_model); self.output_table[f"Hairpin NED"].append(ensemble_defect_nativeHairpin)
        ensemble_defect_nativeTH_Linker = defect(strands=[TH,Linker_GFP], structure=str(mfe_structureTH_Linker[0].structure), model=self.my_model); self.output_table[f"TH+Linker/GFP NED"].append(ensemble_defect_nativeTH_Linker)
        ensemble_defect_nativeSwitch_On = defect(strands=[self.target_sequence,Switch_Off], structure=str(mfe_structureSwitch_On[0].structure), model=self.my_model); self.output_table[f"Switch ON NED"].append(ensemble_defect_nativeSwitch_On)
        ensemble_defect_nativeSwitch_On_GFP = defect(strands=[self.target_sequence,Switch_Off_GFP], structure=str(mfe_structureSwitch_On_GFP[0].structure), model=self.my_model); self.output_table[f"Switch ON-GFP NED"].append(ensemble_defect_nativeSwitch_On_GFP)

    def compute_target_parameters(self,target_sequence):
        """
        Calculates minimum free energy (MFE) and ensemble defects of various regions of the target around each truncated 36nt binding site. 
        Optimally, the target strand is single stranded, indicating an MFE~0 and an ensemble defect ~0
        Args: target_sequence (str):  An RNA sequence consisting of 'A', 'U', 'C', 'G'.
        Returns: appended columns to output_table: A dictionary containing computed thermodynamic values for toehold switch target and various predetermined flanking lengths. 
        """
        mRNA_seq_RNA = self.dna_to_rna(self.mRNA_seq)
        index = mRNA_seq_RNA.find(target_sequence)
        for length in self.flanking_lengths:
            left_flank = max(0, index - length)
            right_flank = min(index + len(self.target_sequence) + length, len(mRNA_seq_RNA))
            flanking_sequence = mRNA_seq_RNA[left_flank:index] + self.target_sequence + mRNA_seq_RNA[index+len(self.target_sequence):right_flank]
            mfe_structures = mfe(strands=[flanking_sequence], model=self.my_model)
            native_structure = mfe_structures[0].structure; ideal_structure = '.' * len(flanking_sequence)
            partition_function = pfunc(strands=[flanking_sequence], model=self.my_model); parfunc = partition_function[1]
            self.output_table[f"Trigger (+-{length} nt) MFE"].append(mfe_structures[0].energy)
            self.output_table[f"Trigger (+-{length} nt) Ensemble Defect Native"].append(defect(strands=[flanking_sequence], structure=str(native_structure), model=self.my_model))
            self.output_table[f"Trigger (+-{length} nt) Ensemble Defect Ideal"].append(defect(strands=[flanking_sequence], structure=ideal_structure, model=self.my_model))

    def compute_codon_fractions(self, mRNA_seq, target_sequence, codon_usage_table):
        mRNA_seq_DNA = mRNA_seq.upper().replace("U", "T"); target_seq = target_sequence.replace("U", "T")  # Convert RNA sequence to DNA (U -> T); Convert target sequence to DNA (U -> T)
        target_start = mRNA_seq_DNA.find(target_seq)  # Find the target sequence in the mRNA sequence
        if target_start == -1:
            raise ValueError("Target sequence not found in mRNA sequence.")
        closest_in_frame_start = target_start - (target_start % 3)  # Adjust to the closest in-frame codon
        codons = [mRNA_seq_DNA[i:i+3] for i in range(0, len(mRNA_seq_DNA) - 2, 3)]  # Extract codons
        codon_fractions = [self.codon_usage_dict.get(c, 0) for c in codons]  # Get codon fractions
        closest_codon_idx = closest_in_frame_start // 3
        target_end_idx = closest_codon_idx + (len(target_seq) // 3)
        # Get fractions of the first two codons
        codon1_fraction = codon_fractions[closest_codon_idx]; codon2_fraction = codon_fractions[closest_codon_idx + 1];  avg_two_codons = (codon1_fraction + codon2_fraction) / 2
        # Compute average codon fraction across the target binding region
        avg_target_region = sum(codon_fractions[closest_codon_idx:target_end_idx]) / (target_end_idx - closest_codon_idx)
        # Compute preceding codon averages
        preceding_averages = {}
        for n in range(4, 11):
            start_idx = max(0, closest_codon_idx - n)
            valid_codon_count = closest_codon_idx - start_idx
            if valid_codon_count > 0:
                avg_value = sum(codon_fractions[start_idx:closest_codon_idx]) / valid_codon_count
            else:
                # If no preceding codons, use the first codon fraction as the default
                avg_value = codon1_fraction
            preceding_averages[f"Preceding {n} codons avg"] = avg_value
            self.output_table[f"Preceding {n} codons avg"].append(avg_value)
        # Compute following codon averages
        following_averages = {}
        for n in range(4, 11):
            end_idx = min(len(codon_fractions), target_end_idx + n)
            valid_codon_count = end_idx - target_end_idx
            if valid_codon_count > 0:
                avg_value = sum(codon_fractions[target_end_idx:end_idx]) / valid_codon_count
            else:
                # If no following codons, use the last codon fraction as the default
                avg_value = codon_fractions[-1]
            following_averages[f"Following {n} codons avg"] = avg_value
            self.output_table[f"Following {n} codons avg"].append(avg_value)
        # Append results to the output table
        self.output_table[f"First Codon Fraction"].append(codon1_fraction)
        self.output_table[f"Second Codon Fraction"].append(codon2_fraction)
        self.output_table[f"Average First Two codons"].append(avg_two_codons)
        self.output_table[f"Average Target Fraction"].append(avg_target_region)
    
    def rank_new_designs(self, new_designs, target_type, model_params_path="all_trained_model_params.pkl"):
        """
        Rank new designs using all five PLS-DA models (OFF AVG, ON AVG Truncated, ON AVG Full, ON OFF Truncated, ON OFF Full) and combine rankings.
        
        Parameters:
            new_designs (np.array or pd.DataFrame): Array or DataFrame of new designs (shape: n_new_designs x n_features).
            model_params_path (str): Path to the saved model parameters. DO NOT CHANGE: always should be all_trained_model_params.pkl
            target_type (str): "Truncated" or "Full" mRNA target; user selects which type to be ranked.
        
        Returns:
            final_ranking (pd.DataFrame): Final ranking of designs with combined scores.
            ranked_indices (np.array): Indices of the ranked designs in the original dataset.
        """
        with open(model_params_path, "rb") as f:  # Load the saved parameters
            model_params = pickle.load(f)
    
        rankings = {}  # Initialize a dictionary to store probabilities for each model

        for model_name, params in model_params.items():  # Compute probabilities for each model
            # Get the selected feature indices for this model
            selected_feature_indices = params["selected_feature_indices"]
    
            # Filter new_designs to include only the selected features (columns)
            if isinstance(new_designs, pd.DataFrame):
                new_designs_filtered = new_designs.iloc[:, selected_feature_indices]  # Filter columns
            else:
                new_designs_filtered = new_designs[:, selected_feature_indices]  # Filter columns
    
            # Filter the scaler parameters to match the selected features
            scaler_mean_filtered = np.mean(new_designs_filtered, axis=0)
            scaler_std_filtered = np.std(new_designs_filtered, axis=0)

            # Standardize new designs using the filtered scaler parameters
            epsilon = 1e-8
            safe_std = np.where(scaler_std_filtered < epsilon, epsilon, scaler_std_filtered)
            new_designs_scaled = (new_designs_filtered - scaler_mean_filtered) / safe_std
            if np.any(scaler_std_filtered < epsilon):
                print(f"Warning: {np.sum(scaler_std_filtered < epsilon)} features had near-zero variance")

            # Transform into latent variables
            new_designs_latent = new_designs_scaled @ params["pls_loadings"]
    
            # Compute decision scores and probabilities
            decision_scores = new_designs_latent @ params["logreg_coefficients"].T
            probabilities = 1 / (1 + np.exp(-decision_scores))

            # Store probabilities for this model
            rankings[model_name] = probabilities
        
        # Combine probabilities based on target type
        if target_type == "Truncated":
            # Use OFF_AVG, ON_AVG_Truncated, and ON_OFF_Truncated
            # In paper, rankings for "OFF AVG" and "ON AVG Truncated" were used in Fig. 4
            combined_scores = (
                #rankings["OFF AVG"]  +
                #rankings["ON AVG Truncated"]  +
                rankings["ON OFF Truncated"] 
            )
        elif target_type == "Full":
            # Use OFF_AVG, ON_AVG_Full, and ON_OFF_Full
            # In paper, rankings for "OFF AVG" and "ON AVG Full" were used in Fig. 4
            combined_scores = (
                #rankings["OFF AVG"]  #+
                #rankings["ON AVG Full"]  #+
                rankings["ON OFF Full"] 
            )
        else:
            raise ValueError("Invalid target_type. Choose 'Truncated' or 'Full'.")

        
        # Rank designs based on combined scores
        ranked_indices = np.argsort(-combined_scores.flatten())  # Negative sign for descending order
        final_ranked_scores = combined_scores[ranked_indices]
    
        # Create a DataFrame for the final ranking
        final_ranking = pd.DataFrame({
            "Rank": range(1, len(ranked_indices) + 1),
            "Combined Score": final_ranked_scores.flatten(),
        })
    
        return final_ranking, ranked_indices        

    def sensor_scoring_VISTA(self, designfile):
        """
        Filters out invalid designs not containing a START codon (AUG), or those that have an in-frame STOP codon.
        
        Parameters:
            mRNA_RR_results_complete (CSV): File containing all computed parameters and riboregulator designs (shape: n_designs x n_features).
    
        Returns:
            {self.design_type}_{self.target_type}_{self.mRNA_name}_all_designs_ranked (CSV): Final ranking of all designs with combined scores.
        """
        design_data = pd.read_csv("mRNA_RR_results_complete.csv")
        #design_data.drop(index=0, inplace=True)
        
        # Extract feature names (excluding the first two columns: 'Target Sequence' and 'Switch Sequence')
        feature_names = [col for i, col in enumerate(design_data.columns) if i not in [0, 1]]
        
        design_spec_data = []
        RBS_seq = 'AGAGGAGA'
        for _, row in design_data.iterrows():
            target_seq = self.dna_to_rna(row['Target Sequence'])
            sensor_seq = self.dna_to_rna(row['Switch Sequence'])
            index_RBS = sensor_seq.find(RBS_seq)
            index_linker = sensor_seq.find(self.hairpin_suffix)
            index_AUG = [i for i in range(len(sensor_seq)) if sensor_seq.startswith('AUG', i)]
            start_codon_index = 0
            for idx in index_AUG:
                if idx < index_RBS:
                    continue
                if (index_linker - idx) % 3 == 0:
                    start_codon_index = idx
                    break
            spec_row = [float(value) for i, value in enumerate(row) if i not in [1, 2]]
            if start_codon_index == 0:
                print(f'***** {self.current_RNA}: Position {spec_row[0]} of {len(self.mRNA_seq)-self.target_len} ERROR: Suitable start codon not found. Continuing... ***** \n')
                spec_row[0] = 0
                design_spec_data.append(spec_row)
                continue
            aa_seq = self.nt_to_aa(sensor_seq[start_codon_index:])
            if '*' in aa_seq:
                print(f'***** {self.current_RNA}: Position {spec_row[0]} of {len(self.mRNA_seq)-self.target_len} ERROR: In-Frame STOP codon. Continuing... ***** \n')
                spec_row[0] = 0
                design_spec_data.append(spec_row)
                continue
            design_spec_data.append(spec_row)

        design_spec_data = np.array(design_spec_data)  # Convert to NumPy array for easier indexing
        
        discard_set = []
        keep_set = []
        # Filter out designs with invalid features--missing start codon or in-frame STOP codon
        for c2 in range(design_spec_data.shape[0]):  # Iterate over rows
            if design_spec_data[c2, 0] == 0:  # Check first element of each row
                discard_set.append(c2)
            else:
                keep_set.append(c2)
        # Keep only valid designs
        filtered_designs = design_spec_data[keep_set, 1:]
        # Rank the filtered designs
        file_name = "all_trained_model_params.pkl"
        full_path = os.path.join(self.path, file_name) # Use os.path.join() to combine the directory path and file name
        final_ranking, ranked_indices = self.rank_new_designs(filtered_designs, self.target_type, full_path)

        # Initialize rank and score columns in design_data
        design_data["Rank"] = np.nan
        design_data["Combined Score"] = np.nan

        # Map ranks and scores back to the original design_data using ranked_indices
        for i, idx in enumerate(ranked_indices):
            original_idx = keep_set[idx]  # Map ranked index back to the original index
            design_data.at[design_data.index[original_idx], "Rank"] = final_ranking.iloc[i]["Rank"]
            design_data.at[design_data.index[original_idx], "Combined Score"] = final_ranking.iloc[i]["Combined Score"]

        # Save the ranked designs to a new CSV file
        design_data.to_csv(f'{self.design_type}_{self.target_type}_{self.current_RNA}_all_designs_ranked.csv', index=False)    

    
    def return_design_object(self): 
        #This function will define the secondary structure of the toehold switch
        os.chdir(self.design_type)
        for c1 in range(len(self.mRNA_names0)):
            self.current_RNA =  self.mRNA_names0.iloc[c1, 0]
            os.mkdir(self.current_RNA);
            os.chdir(self.current_RNA);
            self.mRNA_seq = self.mRNA_sequences0.iloc[c1, 0];
            self.T = float(self.temp.iloc[c1].iloc[0]); #in NUPACK 4, temperature will be changed when specifying a physical model
            #Define a physical model, as NUPACK 4 analysis and design jobs are run based on a physical model created using the Model class:
            self.my_model = Model(material='rna', ensemble='stacking', celsius=self.T,sodium=1.0, magnesium=0.0)

            #Initialize the lenghts of each domain in the canonical toehold switch design
            self.len_a = 3; self.len_b = 3; self.len_c = 30; self.len_d = 3; self.d_dom = 'AAC'; self.toehold_len = self.len_c;
            self.target_len = sum([self.len_a,self.len_b,self.len_c]); self.hairpin_suffix_len = 21; self.output_seq30_len = 30;
    
            self.output_table = {"Position": [], "Target Sequence": [], "Switch Sequence":[]}
            self.output_table[f"Ascending Stem MFE"] = [];self.output_table[f"Descending Stem MFE"] = [];self.output_table[f"RBS-GFP Stem MFE"] = [];self.output_table[f"RBS-Linker MFE"] = [];self.output_table[f"Switch Off MFE"] = [];self.output_table[f"Switch Off GFP MFE"] = [];self.output_table[f"Switch Off NoTH MFE"] = [];self.output_table[f"Hairpin MFE"] = [];self.output_table[f"TH+Linker/GFP MFE"] = [];self.output_table[f"Switch ON MFE"] = [];self.output_table[f"Switch ON-GFP MFE"] = []
            self.output_table[f"Ascending Stem IED"] = [];self.output_table[f"Descending Stem IED"] = [];self.output_table[f"RBS-GFP Stem IED"] = [];self.output_table[f"RBS-Linker IED"] = [];self.output_table[f"Switch Off IED"] = [];self.output_table[f"Switch Off GFP IED"] = [];self.output_table[f"Switch Off NoTH IED"] = [];self.output_table[f"Hairpin IED"] = [];self.output_table[f"TH+Linker/GFP IED"] = [];self.output_table[f"Switch ON IED"] = [];self.output_table[f"Switch ON-GFP IED"] = []
            self.output_table[f"Ascending Stem NED"] = [];self.output_table[f"Descending Stem NED"] = [];self.output_table[f"RBS-GFP Stem NED"] = [];self.output_table[f"RBS-Linker NED"] = [];self.output_table[f"Switch Off NED"] = [];self.output_table[f"Switch Off GFP NED"] = [];self.output_table[f"Switch Off NoTH NED"] = [];self.output_table[f"Hairpin NED"] = [];self.output_table[f"TH+Linker/GFP NED"] = [];self.output_table[f"Switch ON NED"] = [];self.output_table[f"Switch ON-GFP NED"] = []
            self.output_table[f"First Codon Fraction"]= []; self.output_table[f"Second Codon Fraction"]= []; self.output_table[f"Average First Two codons"]= []; self.output_table[f"Average Target Fraction"]= [];
    
            #Here, the flanking lenghts of the 36nt trigger are specified--these are used to compute Ensemble Defect and MFE surrounding the trigger binding site
            self.flanking_lengths=[0,10, 25, 50, 100]
            for length in self.flanking_lengths:
                self.output_table[f"Trigger (+-{length} nt) Ensemble Defect Ideal"] = []; self.output_table[f"Trigger (+-{length} nt) Ensemble Defect Native"] = []; self.output_table[f"Trigger (+-{length} nt) MFE"] = []; 
    
            for n in range(4, 11):
                self.output_table[f"Preceding {n} codons avg"]= []
                self.output_table[f"Following {n} codons avg"]= []

            #Here, we specify the structure of the toehold switch
            self.hairpin_struc_linker_r = 'U%s D%s D3 (U3 D6 U11 U3) U%s U%s'%(self.len_c,(self.len_a+self.len_b),(self.len_d+self.len_a+3),(self.hairpin_suffix_len+self.output_seq30_len))
            self.target_struc = 'U%s'%(self.target_len)
            self.complex_struc_linker_r = 'D%s + U%s D6 U11 U3 D%s U%s U%s'%((self.toehold_len+self.len_a+self.len_b),(9-(self.len_a+self.len_b)), (3+self.len_a), (self.len_b+self.len_d), (self.hairpin_suffix_len+self.output_seq30_len))
            
            #The following code was commented out to compute rankings for mCherry in Figure 4. When designing new toehold switches, this is the script that calls all dependent functions defined above 
            for c2 in range(len(self.mRNA_seq)-self.target_len):
                print('%s: Position %s of %s ... \n'%(self.mRNA_names0.iloc[c1, 0],c2+1, len(self.mRNA_seq)-self.target_len));
                target_seq = self.mRNA_seq[c2:c2+self.target_len];
                target_seq_comp = self.reverse_complement_seq(target_seq);
                toehold = target_seq_comp[:self.toehold_len];
                a_dom = target_seq[:self.len_a];
                b_dom = target_seq[self.len_a:self.len_a+self.len_b];
                a_dom_star = self.reverse_complement_seq(a_dom);
                b_dom_star = self.reverse_complement_seq(b_dom);
                self.output_seq30 = self.output_sequence.iloc[c1, 0][:30]

                hairpin_seq = "".join([toehold,b_dom_star,a_dom_star,self.hairpin_seq_top,a_dom,b_dom,self.d_dom,a_dom_star,reverse_complement(self.hairpin_seq_top[-3:]),self.hairpin_suffix,self.output_seq30]);
                self.sensor_sequence_r =  self.dna_to_rna(hairpin_seq); self.target_sequence = self.dna_to_rna(target_seq);
                self.output_table["Position"].append(c2); self.output_table["Target Sequence"].append(self.target_sequence); self.output_table["Switch Sequence"].append(self.sensor_sequence_r);
                self.compute_sensor_parameters(self.sensor_sequence_r)
                self.compute_target_parameters(self.target_sequence)
                self.compute_codon_fractions(self.mRNA_seq, self.target_sequence, self.codon_usage_dict)
            
            df = pd.DataFrame(self.output_table)
            df.to_csv("mRNA_RR_results_complete.csv", index=False)
            ##End commented out section used in Figure 4
            #sensor_scoring_VISTA used to score toehold switches based on selected thermodynamic and structural features, as selected by RFE prior to PLS_DA
            
            self.sensor_scoring_VISTA("RNA_RR_results_complete.csv")
            
            os.chdir("..")

        os.chdir("..")
        self.format_final_designs(self.num_designs)

    def format_final_designs(self, num_designs):
        """
        Formats the top N designs from the ranked results into the specified structured format.
        
        Args: num_designs (int): Number of top designs to include in the output file (default 6)
                
        Returns: writes a CSV file with designs formatted as specified
        """
        # Initialize output lines
        output_lines = []

        # Create a dictionary mapping mRNA names to sequences
        mrna_dict = dict(zip(self.mRNA_names0['Gene Name'], self.mRNA_sequences0['Gene Sequence']))
        
        # Get the list of RNA directories
        rna_dirs = [d for d in os.listdir(self.design_type) if os.path.isdir(os.path.join(self.design_type, d))]
    
        for rna in rna_dirs:
            rna_path = os.path.join(self.design_type, rna)
            ranked_file = f"{self.design_type}_{self.target_type}_{rna}_all_designs_ranked.csv"
            ranked_path = os.path.join(rna_path, ranked_file)
            
            if os.path.exists(ranked_path):
                # Read the ranked designs file
                df = pd.read_csv(ranked_path)
                
                # Filter out invalid designs (those with Rank = NaN)
                valid_designs = df[df['Rank'].notna()]
                
                # Sort by Rank and take top N designs
                top_designs = valid_designs.sort_values('Rank').head(num_designs)

                # Calculate the average score from the top N combined scores
                avg_score = top_designs['Combined Score'].mean()
                avg_score_str = f"{avg_score:.1f}" if not pd.isna(avg_score) else "nan"
                
                # Get the full target sequence
                full_target_seq = mrna_dict.get(rna, "SEQUENCE_NOT_FOUND")
                
                # Add header information
                self.name_output = self.output_name.iloc[0].iloc[0]
                design_name = f"{rna}_{self.name_output}_T{int(float(self.T))}"
                output_lines.append(f"{design_name} target:,,,")
                output_lines.append("")
                output_lines.append(f"{rna} RNA sequence,{full_target_seq},,,")
                output_lines.append(f"Target region {rna}: Average design score (top {num_designs} devices) =, {avg_score_str},,,")
                output_lines.append("")
                output_lines.append("Sensor name,Index,Sensor RNA sequence,Target RNA subsequence")
                
                # Add each sensor design
                for i, (_, row) in enumerate(top_designs.iterrows(), 1):
                    sensor_name = f"VISTA_{design_name}_{chr(64+i)}"  # A, B, C, etc.
                    sensor_index = row['Position']
                    sensor_seq = self.rna_to_dna(row['Switch Sequence'])
                    target_subseq = self.rna_to_dna(row['Target Sequence'])
                    
                    output_lines.append(f"{sensor_name},{sensor_index},{sensor_seq},{target_subseq}")
                output_lines.append("")
                output_lines.append("")


        # Write to CSV
        output_path = os.path.join(self.design_type, "VISTA_final_design_info.csv")
        
        # Use newline='' to prevent extra blank lines in Windows
        with open(output_path, 'w', newline='') as f:
            f.write('\n'.join(output_lines))
        
        print(f"Successfully saved formatted designs to {output_path}")


In [3]:
# Here, you will call the design class and execute the design process. 

# First, update source_seq_list with your required sequences, output/reporter, and temperature. 

# Under target_type please select whether you would like to optimize for targeting a "Truncated" or "Full" length RNA transcript.

# Num_designs allows you to select the number of toehold switches you would like the code to return. 

# The code will ask you to select a directory. When the window launches, please select the working directory with the necessary files...
# ... for the code to run. These should all be in the toehold-VISTA folder. 


toehold_VISTA = toehold_switch_design_object(
    source_seq_list='source_seq_list.csv', # Update the source_seq_list.csv in the toehold_VISTA folder to include Gene Name, Gene Sequence, Temperature, Output Name, and Output Sequence
    ecoli_codon_usage_table='ecoli_codon_usage_table.csv', # This file should not change--maintain this file in the toehold_VISTA folder for the code to run
    target_type='Full', # Select which toehold switch targeting optimization you desire-- select from "Full" or "Truncated"
    num_designs=12 # Select the number of designs you wish the code to select. 
)
toehold_VISTA.return_design_object()  # Execute the design process

2025-08-12 13:11:07.241 python[61988:2887316] +[CATransaction synchronize] called within transaction


ryhB_sRNA: Position 1 of 54 ... 

ryhB_sRNA: Position 2 of 54 ... 

ryhB_sRNA: Position 3 of 54 ... 

ryhB_sRNA: Position 4 of 54 ... 

ryhB_sRNA: Position 5 of 54 ... 

ryhB_sRNA: Position 6 of 54 ... 

ryhB_sRNA: Position 7 of 54 ... 

ryhB_sRNA: Position 8 of 54 ... 

ryhB_sRNA: Position 9 of 54 ... 

ryhB_sRNA: Position 10 of 54 ... 

ryhB_sRNA: Position 11 of 54 ... 

ryhB_sRNA: Position 12 of 54 ... 

ryhB_sRNA: Position 13 of 54 ... 

ryhB_sRNA: Position 14 of 54 ... 

ryhB_sRNA: Position 15 of 54 ... 

ryhB_sRNA: Position 16 of 54 ... 

ryhB_sRNA: Position 17 of 54 ... 

ryhB_sRNA: Position 18 of 54 ... 

ryhB_sRNA: Position 19 of 54 ... 

ryhB_sRNA: Position 20 of 54 ... 

ryhB_sRNA: Position 21 of 54 ... 

ryhB_sRNA: Position 22 of 54 ... 

ryhB_sRNA: Position 23 of 54 ... 

ryhB_sRNA: Position 24 of 54 ... 

ryhB_sRNA: Position 25 of 54 ... 

ryhB_sRNA: Position 26 of 54 ... 

ryhB_sRNA: Position 27 of 54 ... 

ryhB_sRNA: Position 28 of 54 ... 

ryhB_sRNA: Position 29 of 54 